# Data Engineering EDA Project

By Aommy, Austin, Ling, Tath (yippee)

-----------------------

## Installing Dependencies

### Minio Object Store dependencies

1. Install minio cli if not already installed.
2. Create client
3. Make bucket (if needed)

In [10]:
! pip install minio --quiet

In [2]:
from minio import Minio

# Create client with access and secret key.
# client = Minio("s3.amazonaws.com", "ACCESS-KEY", "SECRET-KEY")

# Create client with access key and secret key with specific region.
client = Minio(
    "localhost:9000",
    access_key="ROOTNAME",
    secret_key="CHANGEME123",
    secure=False
)

if client.bucket_exists("eda"):
    print("Bucket exists.")
else:
    client.make_bucket("eda")
    print("Bucket created.")

Bucket exists.


## Importing libraries

In [23]:
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt

import io
from urllib.request import urlopen

In [7]:
!pip install --quiet yfinance --upgrade --no-cache-dir

---------------------

## Data Collection

### Data from Global Events

Installing data from http://data.gdeltproject.org/events/index.html for global event data!

In [5]:
for y in range(19, 25):
    for m in range(1, 13):
        if m < 10:
            m = "0" + str(m)
        for d in range(1, 32):
            if d < 10:
                d = "0" + str(d)
            try:
                response = client.get_object("eda", "gdelt/20{0}/{1}/{2}.export.CSV.zip".format(y,m,d))
                # Read data from response.
            except:
                print("File doesn't exist!")
                try:
                    data = urlopen(
                        "http://data.gdeltproject.org/events/20{0}{1}{2}.export.CSV.zip".format(y,m,d),
                    )
                    print("Uploading file!")
                    result = client.put_object(
                        "eda", "gdelt/20{0}/{1}/{2}.export.CSV.zip".format(y,m,d), data, length=-1, part_size=10*1024*1024,
                    )
                    print(
                        "created {0} object; etag: {1}, version-id: {2}".format(
                            result.object_name, result.etag, result.version_id,
                        ),
                    )
                except:
                    print("No file to download!")
            finally:
                response.close()
                response.release_conn()

File doesn't exist!
No file to download!
File doesn't exist!
No file to download!
File doesn't exist!
No file to download!
File doesn't exist!
No file to download!
File doesn't exist!
No file to download!
File doesn't exist!
No file to download!
File doesn't exist!
No file to download!
File doesn't exist!
No file to download!
File doesn't exist!
No file to download!
File doesn't exist!
Uploading file!
created gdelt/2020/04/09.export.CSV.zip object; etag: 5cf7553a7e25189741f5f55711b3a5f2, version-id: None
File doesn't exist!
No file to download!
File doesn't exist!
No file to download!
File doesn't exist!
No file to download!
File doesn't exist!
No file to download!
File doesn't exist!
No file to download!
File doesn't exist!
No file to download!
File doesn't exist!
No file to download!
File doesn't exist!
No file to download!
File doesn't exist!
No file to download!
File doesn't exist!
No file to download!
File doesn't exist!
No file to download!
File doesn't exist!
No file to download

### Data from YahooFinance

In [30]:
import yfinance as yf

companies = {
    "tech": ['AAPL', 'MSFT', 'NVDA'],
    "energy": ['XOM', 'CVX', 'BP', 'COP'],
    "food": ['KO', 'PEP', 'PG'],
    "health": ['JNJ', 'PFE', 'ABBV'],
    "finance": ['JPM', 'V', 'MA'],
    "agriculture": ['DE', 'CTVA', 'ADM'],
    "telecom": ['T', 'VZ', 'TMUS'],
    "materials": ['DOW', 'NEM', 'FCX']
}

In [37]:
uploaded = False
for sector in companies:
    for stock in companies[sector]:
        try:
            response = client.get_object("eda", "yfinance/{0}/{1}.snappy.parquet".format(sector,stock))
            # Read data from response.
        except:
            print("File doesn't exist!")
            data = yf.download(stock, start="2019-01-01", end="2024-10-15", progress=False)
            result = client.put_object(
                "eda", "yfinance/{0}/{1}.snappy.parquet".format(sector,stock), io.BytesIO(data.to_parquet(compression='snappy')), length=-1, part_size=10*1024*1024,
            )
            print(
                "created {0} object; etag: {1}, version-id: {2}".format(
                    result.object_name, result.etag, result.version_id,
                ),
            )
            uploaded = True
        finally:
            response.close()
            response.release_conn()
if (not uploaded):
    print("No changes to make.")

No changes to make.


----------------------

## Data Processing

In [50]:
# resp = client.get_object("eda", "gdelt/2019/01/01.export.CSV.zip")
# from io import BytesIO

# df = pd.read_csv(BytesIO(resp.data))

In [49]:
# df = pd.read_csv(urlopen("http://data.gdeltproject.org/events/20{0}{1}{2}.export.CSV.zip".format(19,'01','01'),))

----------------------

## Data Analysis